In [2]:
import pandas as pd
import numpy as np
import MySQLdb
import os
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np

In [3]:
db = MySQLdb.connect(host='104.154.94.28',db='loganp',\
                     read_default_file="~/.my.cnf",\
                     autocommit=True,\
                     local_infile = 1)
c=db.cursor()

In [4]:
def get_targets(ra,dec,ang_diam,project,name,dist=5000,con=db,database='master_gaia_database'):
    """
    Given an RA/Dec array of any size, returns a Pandas databse of
    the targets from the master database that fall within 
    a circle of specified angular diameter of the given RA/Decs
    All angles in degrees
    Args:
        ra,dec (array, float [deg]): arrays of pointing coordinates in decimal degrees
        ang_diam (float [deg]): angular size of the diameter of the beam you are simulating.  Default of 0.8 deg
                          is slightly smaller than the MeerKAT beam in L-band to provide a conservative estimate
        project (string): project name that this pointing came from
        name (string): name of the primary beam target
        dist (float, [pc]): depth of the desired query in parsecs.  Default is set larger than the largest distance
                          in the master database to return essentially no distance cut
        con (MySQL connection object): the connection you would like to use for the retrieval
        database (str): the database to query.  Default is the master database of all Gaia targets in our program
    Returns:
        Pandas dataframe of every object in the database meeting criteria
    """
    index = range(len(ra))
    appended_data = [] #make a list to store dataframes
    for r,d,i in zip(ra,dec,index):
        string = 'SELECT * FROM '+str(database)+' \
                    WHERE POWER((ra-('+str(r)+')),2) + POWER((decl - ('+str(d)+')),2) < '+str((ang_diam/2.)**2)+' \
                    AND `dist.c` <= '+str(dist)+';'
        dataframe = pd.read_sql(string, con=con)
        dataframe['project']=project
        dataframe['name']=name
        # store DataFrame in list
        appended_data.append(dataframe)
        print "I've done ",i+1," of ",len(ra)," total pointings"
    targets = pd.concat(appended_data, axis=0)
    return targets

Fermi sources come from here: https://fermi.gsfc.nasa.gov/ssc/data/access/lat/4yr_catalog/3FGL-table/#aitoff <br>

In [5]:
k=pd.read_table('data.txt',delimiter=',')
k

,Source Name,Association,RA,Dec,Gal l,Gal b,Flux 1-100 GeV(photons cm-2s-1),+/-,Spectral Index,+/-.1,Variability Index,Classification,Light Curve(Likelihood),Spectrum,DataAccess
0,3FGL J0000.1+6545,NaN,0.0377,65.7517,117.6940,3.402960,1.015960e-09,1.575920e-10,2.41096,0.082321,40.7535,Unassociated,Link,Link,Link
1,3FGL J0000.2-3738,NaN,0.0612,-37.6484,345.4110,-74.946800,2.038660e-10,5.422530e-11,1.86673,0.179395,44.8453,Unassociated,Link,Link,Link
2,3FGL J0001.6+3535,NaN,0.4043,35.5905,111.6610,-26.188500,2.681200e-10,6.930390e-11,2.35052,0.185889,32.3943,Unassociated,Link,Link,Link
3,3FGL J0002.0-6722,NaN,0.5240,-67.3703,310.1390,-49.061800,2.921170e-10,6.366240e-11,1.94606,0.158511,46.1250,Unassociated,Link,Link,Link
4,3FGL J0002.6+6218,NaN,0.6745,62.3006,117.3020,-0.037116,2.844880e-09,2.108390e-10,2.08626,0.111090,48.0235,Unassociated,Link,Link,Link
5,3FGL J0003.4+3100,NaN,0.8576,31.0085,110.9640,-30.745100,3.139310e-10,6.983020e-11,2.55384,0.130900,38.5360,Unassociated,Link,Link,Link
6,3FGL J0003.5+5721,NaN,0.8903,57.3597,116.4860,-4.911600,4.716470e-10,9.654750e-11,2.18306,0.125800,31.1214,Unassociated,Link,Link,Link
7,3FGL J0004.2+6757,NaN,1.0551,67.9593,118.5090,5.494000,8.158670e-10,1.398660e-10,2.49684,0.093862,34.8146,Unassociated,Link,Link,Link
8,3FGL J0004.2+0843,NaN,1.0574,8.7274,103.5990,-52.363300,2.683930e-10,7.166100e-11,2.05044,0.190928,45.3317,Unassociated,Link,Link,Link
9,3FGL J0006.2+0135,NaN,1.5718,1.5853,100.4010,-59.296900,2.991090e-10,6.773840e-11,2.18690,0.167244,45.5663,Unassociated,Link,Link,Link


In [6]:
# According to Scott Ransom, they will target unassociated sources that are more than 2 deg outside the galactic plane:
m=k.loc[np.where(np.abs(k['Gal b'].values)>2)[0]]
m=m.reset_index(drop=True)
m

,Source Name,Association,RA,Dec,Gal l,Gal b,Flux 1-100 GeV(photons cm-2s-1),+/-,Spectral Index,+/-.1,Variability Index,Classification,Light Curve(Likelihood),Spectrum,DataAccess
0,3FGL J0000.1+6545,NaN,0.0377,65.7517,117.69400,3.40296,1.015960e-09,1.575920e-10,2.41096,0.082321,40.7535,Unassociated,Link,Link,Link
1,3FGL J0000.2-3738,NaN,0.0612,-37.6484,345.41100,-74.94680,2.038660e-10,5.422530e-11,1.86673,0.179395,44.8453,Unassociated,Link,Link,Link
2,3FGL J0001.6+3535,NaN,0.4043,35.5905,111.66100,-26.18850,2.681200e-10,6.930390e-11,2.35052,0.185889,32.3943,Unassociated,Link,Link,Link
3,3FGL J0002.0-6722,NaN,0.5240,-67.3703,310.13900,-49.06180,2.921170e-10,6.366240e-11,1.94606,0.158511,46.1250,Unassociated,Link,Link,Link
4,3FGL J0003.4+3100,NaN,0.8576,31.0085,110.96400,-30.74510,3.139310e-10,6.983020e-11,2.55384,0.130900,38.5360,Unassociated,Link,Link,Link
5,3FGL J0003.5+5721,NaN,0.8903,57.3597,116.48600,-4.91160,4.716470e-10,9.654750e-11,2.18306,0.125800,31.1214,Unassociated,Link,Link,Link
6,3FGL J0004.2+6757,NaN,1.0551,67.9593,118.50900,5.49400,8.158670e-10,1.398660e-10,2.49684,0.093862,34.8146,Unassociated,Link,Link,Link
7,3FGL J0004.2+0843,NaN,1.0574,8.7274,103.59900,-52.36330,2.683930e-10,7.166100e-11,2.05044,0.190928,45.3317,Unassociated,Link,Link,Link
8,3FGL J0006.2+0135,NaN,1.5718,1.5853,100.40100,-59.29690,2.991090e-10,6.773840e-11,2.18690,0.167244,45.5663,Unassociated,Link,Link,Link
9,3FGL J0006.6+4618,NaN,1.6512,46.3054,114.90800,-15.86660,3.095000e-10,7.406530e-11,2.41726,0.188381,48.8199,Unassociated,Link,Link,Link


In [7]:
# But that is still 800 sources.  They will be shooting for 100-200.  So Scott also says the ones most likely to be 
# pulsars will have a low variability index, and thus more likely to be targeted.  He provided a plot to illustrate
# (included in this folder) that shows psr like objects are less than about 40.

n=m.loc[np.where(m['Variability Index'].values<40)[0]]
n=n.reset_index(drop=True)
n

,Source Name,Association,RA,Dec,Gal l,Gal b,Flux 1-100 GeV(photons cm-2s-1),+/-,Spectral Index,+/-.1,Variability Index,Classification,Light Curve(Likelihood),Spectrum,DataAccess
0,3FGL J0001.6+3535,NaN,0.4043,35.5905,111.6610,-26.18850,2.681200e-10,6.930390e-11,2.35052,0.185889,32.3943,Unassociated,Link,Link,Link
1,3FGL J0003.4+3100,NaN,0.8576,31.0085,110.9640,-30.74510,3.139310e-10,6.983020e-11,2.55384,0.130900,38.5360,Unassociated,Link,Link,Link
2,3FGL J0003.5+5721,NaN,0.8903,57.3597,116.4860,-4.91160,4.716470e-10,9.654750e-11,2.18306,0.125800,31.1214,Unassociated,Link,Link,Link
3,3FGL J0004.2+6757,NaN,1.0551,67.9593,118.5090,5.49400,8.158670e-10,1.398660e-10,2.49684,0.093862,34.8146,Unassociated,Link,Link,Link
4,3FGL J0008.3+1456,NaN,2.0881,14.9336,107.6790,-46.65810,4.115650e-10,7.778280e-11,2.26955,0.128346,34.4397,Unassociated,Link,Link,Link
5,3FGL J0008.5+6853,NaN,2.1409,68.8870,119.0670,6.33763,1.045430e-09,1.806540e-10,2.42290,0.229519,35.0343,Unassociated,Link,Link,Link
6,3FGL J0017.1+1445,NaN,4.2937,14.7612,110.6720,-47.29330,2.452750e-10,6.938830e-11,1.88303,0.177631,36.1434,Unassociated,Link,Link,Link
7,3FGL J0022.7+4651,NaN,5.6872,46.8528,117.8430,-15.73520,1.882530e-10,6.253390e-11,2.78427,0.192955,38.1622,Unassociated,Link,Link,Link
8,3FGL J0031.2-2320,NaN,7.8130,-23.3427,71.5023,-84.07110,2.372560e-10,6.107380e-11,2.13911,0.170399,33.5063,Unassociated,Link,Link,Link
9,3FGL J0031.6+0938,NaN,7.9225,9.6478,114.8450,-52.91350,2.345140e-10,6.168880e-11,2.65719,0.163768,28.8198,Unassociated,Link,Link,Link


In [8]:
n=n.loc[np.where(n['Dec'].values<45.)[0]]
n=n.reset_index(drop=True)
n

,Source Name,Association,RA,Dec,Gal l,Gal b,Flux 1-100 GeV(photons cm-2s-1),+/-,Spectral Index,+/-.1,Variability Index,Classification,Light Curve(Likelihood),Spectrum,DataAccess
0,3FGL J0001.6+3535,NaN,0.4043,35.5905,111.66100,-26.18850,2.681200e-10,6.930390e-11,2.35052,0.185889,32.3943,Unassociated,Link,Link,Link
1,3FGL J0003.4+3100,NaN,0.8576,31.0085,110.96400,-30.74510,3.139310e-10,6.983020e-11,2.55384,0.130900,38.5360,Unassociated,Link,Link,Link
2,3FGL J0008.3+1456,NaN,2.0881,14.9336,107.67900,-46.65810,4.115650e-10,7.778280e-11,2.26955,0.128346,34.4397,Unassociated,Link,Link,Link
3,3FGL J0017.1+1445,NaN,4.2937,14.7612,110.67200,-47.29330,2.452750e-10,6.938830e-11,1.88303,0.177631,36.1434,Unassociated,Link,Link,Link
4,3FGL J0031.2-2320,NaN,7.8130,-23.3427,71.50230,-84.07110,2.372560e-10,6.107380e-11,2.13911,0.170399,33.5063,Unassociated,Link,Link,Link
5,3FGL J0031.6+0938,NaN,7.9225,9.6478,114.84500,-52.91350,2.345140e-10,6.168880e-11,2.65719,0.163768,28.8198,Unassociated,Link,Link,Link
6,3FGL J0032.5+3912,NaN,8.1464,39.2131,118.95100,-23.51280,2.375060e-10,6.068670e-11,2.56391,0.166556,35.1931,Unassociated,Link,Link,Link
7,3FGL J0049.0+4224,NaN,12.2530,42.4130,122.45500,-20.45650,2.613640e-10,6.981640e-11,1.76537,0.166987,35.6061,Unassociated,Link,Link,Link
8,3FGL J0112.9-7506,NaN,18.2438,-75.1053,301.07400,-41.94520,2.921530e-10,6.485800e-11,2.17215,0.170934,39.5860,Unassociated,Link,Link,Link
9,3FGL J0114.8+1917,NaN,18.7099,19.2947,130.52000,-43.23190,2.331130e-10,6.069340e-11,2.31024,0.230573,38.4403,Unassociated,Link,Link,Link


In [9]:
# That's 217, that's a good number to draw from.
n.to_csv('Trapum_fermi_targets.csv')